# LeafDieseaseDetection

In [1]:
#importing Neccessary Libraries
import os 
import numpy as np
from PIL import Image

In [2]:
data_path = 'image data/ '
fruit = 'apple'

In [3]:
# Creating Train Data Path for a single Fruit
train_data_path = os.path.join(data_path,"train",fruit)

In [4]:
def load_image_dataset(dataset_directory:str,img_height:int = 224,img_width:int = 224,flatten_image:bool = False):
    """ 
        Params:
        dataset_directory: path to your dataset directory,
        img_height = standard height of image
        img_width = standard width of image
        flatten_image = False: Image will be returned with actual size 
                        True : Image will be flattened 
         
    """   

    # Create empty lists to store the images and their corresponding labels
    images = []
    labels = []

    # Iterate through the subdirectories (each subdirectory represents a class)
    for class_name in os.listdir(dataset_directory):
        class_dir = os.path.join(dataset_directory, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = Image.open(image_path)  # Open the image using PIL
            image = image.resize((img_width, img_height))  # Resize the image to a desired size
            image = np.array(image)  # Convert the image to a numpy array
            images.append(image)
            labels.append(class_name)

    # Convert the lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    
    if flatten_image: # flattening image i.e converting image into vector
        images = images.reshape(images.shape[0],-1)

    return images,labels


In [5]:
train_images,train_labels = load_image_dataset(dataset_directory=train_data_path,flatten_image=True)

In [6]:
train_images.shape

(2286, 150528)

In [7]:
train_labels.shape

(2286,)

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
RFC = RandomForestClassifier(verbose=True)

In [10]:
RFC.fit(X=train_images,y=train_labels)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   51.3s


RandomForestClassifier(verbose=True)

##### Saving trained model

In [11]:
import joblib

def save_model(model,fruit:str,filename:str,location:str):

    location = os.path.join(location,fruit)
    if not os.path.isdir(location):
        os.mkdir(location)
        
    filename = os.path.join(location,filename+".jblib")

    joblib.dump(model,filename=filename)

In [12]:
save_model(RFC,fruit=fruit,location="models/",filename="RFC")

## Testing Process

##### loading saved model

In [13]:
RFC = joblib.load('models/apple/RFC.jblib') # Loaded the saved model

In [14]:
test_data_path = os.path.join(data_path,"test",fruit)

In [15]:
test_images,test_labels =  load_image_dataset(dataset_directory=test_data_path,flatten_image=True)

In [16]:
prediction = RFC.predict(test_images)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
confusion_matrix(y_true=test_labels,y_pred=prediction)

array([[ 63,   0,   0,   0],
       [  0,  62,   0,   0],
       [  0,   0,  27,   0],
       [  0,   0,   0, 164]], dtype=int64)